In [31]:
import cv2
import os, glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rembg import remove

In [32]:
# 데이터 관련 경로
folder_path = 'C:\\BIG_DATA_\\Opencv\\TF_waste\\can' # 사진 파일들이 있는 폴더 경로
file_path_list = os.listdir(folder_path)

new_path = "C:\\BIG_DATA_\\Opencv\\TF_waste\\can\\canCSV"
cnt=1
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        date_part = filename.split("_")[0]  # 날짜 부분 추출
        new_filename = f"can_{cnt:<2}.jpg"  # 새로운 파일명 생성
        
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, new_filename)
        
        os.rename(old_path, new_path)  # 파일 이름 변경
        cnt+=1

In [33]:
def imgRotate(arr):
    arr = arr.reshape((150, 150))
    image_list = []

    height, width = arr.shape[:2]
    # 회전 변환 행렬 반환
    for degree in range(0, 360, 15):
        matrix = cv2.getRotationMatrix2D((width/2, height/2), degree, 1)

        # 회전 변환행렬에 따른 회전 이미지 반환
        rotated_image = cv2.warpAffine(arr, matrix, (width, height))
        image_list.append(rotated_image.flatten())
    final_img_array = np.array(image_list)
    return final_img_array 

In [34]:
def GausianBlur(arr):
    blurred_img = cv2.GaussianBlur(arr, (5, 5), 0)

In [35]:
final_df = pd.DataFrame([])

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        resized_img = cv2.resize(img, (150, 150))

        df = pd.DataFrame(resized_img.flatten()).T  # 이미지 배열을 1차원으로 변환하고 행 벡터로 전치
        final_df = pd.concat([final_df, df]) 

        rotate_df = pd.DataFrame(imgRotate(resized_img))
        final_df = pd.concat([final_df, df]) 
final_df.reset_index(drop=True, inplace=True)

In [36]:
final_df.shape

(408, 22500)

In [37]:
final_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,22490,22491,22492,22493,22494,22495,22496,22497,22498,22499
403,177,179,183,183,185,182,182,181,174,176,...,179,178,179,180,180,180,185,180,184,183
404,158,165,162,147,155,160,167,155,145,151,...,163,175,170,181,177,171,179,178,183,179
405,158,165,162,147,155,160,167,155,145,151,...,163,175,170,181,177,171,179,178,183,179
406,191,191,186,189,186,191,188,179,181,179,...,186,185,187,187,188,189,184,181,181,180
407,191,191,186,189,186,191,188,179,181,179,...,186,185,187,187,188,189,184,181,181,180


In [38]:
# type(final_df.iloc[0, :]), final_df.iloc[0, :].shape

In [39]:
rotated_images_df = pd.DataFrame([])

for index, row in final_df.iterrows():
    row_array = row.values  # 행 데이터를 numpy 배열로 변환
    rotated_images= imgRotate(row_array)
    
    for r in rotated_images:
        rotate_df=pd.DataFrame(r).T
        rotated_images_df=pd.concat([rotated_images_df ,rotate_df])  
    
rotated_images_df.shape

(9792, 22500)

In [40]:
# 확인용

# rotate_test = rotated_images_df.iloc[0, :].to_numpy().reshape(150, 150, 1)
# cv2.imshow('rotate_test', rotate_test)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [41]:
blurred_images_df = pd.DataFrame([])

for index, row in final_df.iterrows():
    row_array = row.values  # 행 데이터를 numpy 배열로 변환
    image = row_array.reshape((150,150))
    blurred_image = cv2.GaussianBlur(image,(3 ,3),5)
    
    blur_df = pd.DataFrame(blurred_image.flatten()).T 
    blurred_images_df = pd.concat([blurred_images_df, blur_df], ignore_index=True)  
    
blurred_images_df.shape

(408, 22500)

In [42]:
# 확인용

# blurred_test = blurred_images_df.iloc[0, :].to_numpy().reshape(150, 150, 1)
# cv2.imshow('blurred_test', blurred_test)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [43]:
sharp_images_df = pd.DataFrame([])
kernel = np.array([[0, -1, 0], 
                   [-1, 5,-1], 
                   [0, -1, 0]])

for index,row in final_df.iterrows():
    row_array=row.values 
    image=row_array.reshape((150 ,150)) 

    # 필터 적용하기
    sharp_image=cv2.filter2D(image,-1,kernel) 

    df=pd.DataFrame(sharp_image.flatten()).T 
    sharp_images_df=pd.concat([sharp_images_df ,df], ignore_index=True)
sharp_images_df.shape

(408, 22500)

In [44]:
# 확인용

# sharped_test = sharp_images_df.iloc[0, :].to_numpy().reshape(150, 150, 1)
# cv2.imshow('sharped_test', sharped_test)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [45]:
final_ = pd.concat([final_df, rotated_images_df, blurred_images_df, sharp_images_df], ignore_index=True)

In [46]:
final_['label'] = 'can'

In [47]:
final_.shape

(11016, 22501)

In [48]:
final_.head(10)

,0,1,2,3,4,5,6,7,8,9,...,22491,22492,22493,22494,22495,22496,22497,22498,22499,label
0,200,202,202,208,202,203,208,200,202,200,...,177,178,192,197,183,191,202,188,191,can
1,200,202,202,208,202,203,208,200,202,200,...,177,178,192,197,183,191,202,188,191,can
2,205,196,200,205,203,203,207,199,203,202,...,135,126,132,139,133,127,133,131,125,can
3,205,196,200,205,203,203,207,199,203,202,...,135,126,132,139,133,127,133,131,125,can
4,156,156,155,163,164,160,170,161,166,159,...,218,224,220,221,216,217,215,215,216,can
5,156,156,155,163,164,160,170,161,166,159,...,218,224,220,221,216,217,215,215,216,can
6,138,142,143,145,143,148,150,146,147,146,...,120,120,121,121,122,122,122,124,125,can
7,138,142,143,145,143,148,150,146,147,146,...,120,120,121,121,122,122,122,124,125,can
8,172,167,167,175,172,173,184,179,179,185,...,217,220,216,217,224,225,221,218,221,can
9,172,167,167,175,172,173,184,179,179,185,...,217,220,216,217,224,225,221,218,221,can


In [49]:
final_.head()

,0,1,2,3,4,5,6,7,8,9,...,22491,22492,22493,22494,22495,22496,22497,22498,22499,label
0,200,202,202,208,202,203,208,200,202,200,...,177,178,192,197,183,191,202,188,191,can
1,200,202,202,208,202,203,208,200,202,200,...,177,178,192,197,183,191,202,188,191,can
2,205,196,200,205,203,203,207,199,203,202,...,135,126,132,139,133,127,133,131,125,can
3,205,196,200,205,203,203,207,199,203,202,...,135,126,132,139,133,127,133,131,125,can
4,156,156,155,163,164,160,170,161,166,159,...,218,224,220,221,216,217,215,215,216,can


In [50]:
final_.tail()

,0,1,2,3,4,5,6,7,8,9,...,22491,22492,22493,22494,22495,22496,22497,22498,22499,label
11011,161,171,183,181,194,177,187,191,157,161,...,176,175,179,164,161,189,157,179,177,can
11012,130,189,200,116,148,154,218,157,103,139,...,214,148,230,175,145,190,168,180,165,can
11013,130,189,200,116,148,154,218,157,103,139,...,214,148,230,175,145,190,168,180,165,can
11014,193,198,172,193,178,201,202,172,179,182,...,178,187,182,188,195,180,178,180,178,can
11015,193,198,172,193,178,201,202,172,179,182,...,178,187,182,188,195,180,178,180,178,can


In [51]:
# 최종 확인용 (원본)

# origin_idx = final_df.shape[0] -1 # 원본 A
# rotate_idx = origin_idx + (rotated_images_df.shape[0]-1) # 원본 A의 회전 인덱스
# blurr_idx = rotate_idx + (blurred_images_df.shape[0]-1) # 원본 A의 블러 인덱스
# sharpen_idx = blurr_idx + (sharp_images_df.shape[0]-1) # 원본 A의 샤픈 인덱스

# original = final_.iloc[origin_idx, :-1].to_numpy().reshape(150, 150, 1)
# original = original.astype(np.uint8) 
# cv2.imshow('original', original)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [52]:
# 최종 확인용 (회전)

# rotate = final_.iloc[rotate_idx, :-1].to_numpy().reshape(150, 150, 1)
# rotate = rotate.astype(np.uint8) 
# cv2.imshow('rotate', rotate)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [53]:
# 최종 확인용 (블러)

# blurr = final_.iloc[blurr_idx, :-1].to_numpy().reshape(150, 150, 1)
# blurr = blurr.astype(np.uint8) 

# cv2.imshow('blurr', blurr)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [54]:
# 최종 확인용 (샤프)

# sharp = final_.iloc[sharpen_idx, :-1].to_numpy().reshape(150, 150, 1)
# sharp = sharp.astype(np.uint8) 
# cv2.imshow('sharp', sharp)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [55]:
final_.to_csv('can_all.csv', index=False)